---

<!-- <a href="https://github.com/rraadd88/roux/blob/master/examples/roux_workflow_cfgs.ipynb"><img align="right" style="float:right;" src="https://img.shields.io/badge/-source-cccccc?style=flat-square"></a>
 -->
 
## ⚙️🗺️ Reading multiple configs.

In [1]:
# install extra requirements
# !pip install roux[workflow]

import logging
logging.basicConfig(level='INFO',force=True)

### Demo inputs

In [2]:
input_dir_path='tests/input/roux_workflow_cfgs/'

In [3]:
## creating the demo configs
configs=dict(
    ## meta-config that contains other configs
    meta_config="""
    key1: value1
    key2: value2
    key3: interpolated from metaconfig = ${key2}
    
    ## append other configs
    sub_config1:
        config_path: '"""+input_dir_path+"""sub_config1.yaml' # should be one key
        path: path/to/file1
        description: sample1. 
    sub_config2:
        config_path: '"""+input_dir_path+"""sub_config2.yaml' # path should exist
        path: path/to/file2
        description: sample2. 
        
    sub_config3:
        config_base_path: '"""+input_dir_path+"""sub_config_base.yaml' # path should exist    
        input_path: ${..sub_config2.path}        
    """,
    ## individual configs to be appended
    sub_config1="""
    c1key1: value from metaconfig = ${key1} # interpolated from c0
    c1key2: value2
    c1key3: value3
    """,

    sub_config2="""
    c2key1: value from metaconfig = ${key1} # interpolated from c0
    c2key2: value interpolated in config1 = ${sub_config1.c1key1} # interpolated from c1
    c2key3: value3
    """,
    
    sub_config_base="""
    input_path: input path
    output_path: output path
    """
    )
import yaml
configs={k:yaml.safe_load(configs[k]) for k in configs}
from pprint import pprint
pprint({k:d for k,d in configs.items()})

{'meta_config': {'key1': 'value1',
                 'key2': 'value2',
                 'key3': 'interpolated from metaconfig = ${key2}',
                 'sub_config1': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
                                 'description': 'sample1.',
                                 'path': 'path/to/file1'},
                 'sub_config2': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
                                 'description': 'sample2.',
                                 'path': 'path/to/file2'},
                 'sub_config3': {'config_base_path': 'tests/input/roux_workflow_cfgs/sub_config_base.yaml',
                                 'input_path': '${..sub_config2.path}'}},
 'sub_config1': {'c1key1': 'value from metaconfig = ${key1}',
                 'c1key2': 'value2',
                 'c1key3': 'value3'},
 'sub_config2': {'c2key1': 'value from metaconfig = ${key1}',
                 'c2key2': 'value interpolated

In [4]:
## save the demo configs
from roux.lib.io import to_dict
[to_dict(configs[k],f'tests/input/roux_workflow_cfgs/{k}.yaml') for k in configs]

['tests/input/roux_workflow_cfgs/meta_config.yaml',
 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
 'tests/input/roux_workflow_cfgs/sub_config_base.yaml']

## Subs

In [5]:
%run ../roux/workflow/cfgs.py
# from roux.workflow.cfgs import read_sub_configs
read_sub_configs(
    configs['meta_config'],
    # config_path_key: str = "config_path",
    # config_base_path_key: str = "config_base_path",    
    verbose=True,
    )

info:root:Appending config to sub_config1
info:root:base config used.
info:root:Appending config to sub_config2
info:root:base config used.
info:root:Appending config to base from sub_config3


{'key1': 'value1',
 'key2': 'value2',
 'key3': 'interpolated from metaconfig = value2',
 'sub_config1': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
  'path': 'path/to/file1',
  'description': 'sample1.',
  'c1key1': 'value from metaconfig = value1',
  'c1key2': 'value2',
  'c1key3': 'value3'},
 'sub_config2': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
  'path': 'path/to/file2',
  'description': 'sample2.',
  'c2key1': 'value from metaconfig = value1',
  'c2key2': 'value interpolated in config1 = value from metaconfig = value1',
  'c2key3': 'value3'},
 'sub_config3': {'input_path': 'path/to/file2', 'output_path': 'output path'}}

## Mta-config

In [6]:
%run ../roux/workflow/cfgs.py
# from roux.workflow.cfgs import read_metadata 
config=read_metadata(
    f'{input_dir_path}/meta_config.yaml', # path to the meta-config
    verbose=True, # verbose
    )
config

info:root:Appending config to sub_config1
info:root:base config used.
info:root:Appending config to sub_config2
info:root:base config used.
info:root:Appending config to base from sub_config3


{'key1': 'value1',
 'key2': 'value2',
 'key3': 'interpolated from metaconfig = value2',
 'sub_config1': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
  'path': 'path/to/file1',
  'description': 'sample1.',
  'c1key1': 'value from metaconfig = value1',
  'c1key2': 'value2',
  'c1key3': 'value3'},
 'sub_config2': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
  'path': 'path/to/file2',
  'description': 'sample2.',
  'c2key1': 'value from metaconfig = value1',
  'c2key2': 'value interpolated in config1 = value from metaconfig = value1',
  'c2key3': 'value3'},
 'sub_config3': {'input_path': 'path/to/file2', 'output_path': 'output path'}}

In [7]:
## test for testbook
assert config['sub_config2']['c2key2']=='value interpolated in config1 = value from metaconfig = value1', config['sub_config2']['c2key2']
assert len(config)==6, len(config)
assert len(config['sub_config2'])==6, config['sub_config2']
print(config['sub_config2']['c2key2'])

value interpolated in config1 = value from metaconfig = value1


### Modifying config using `inputs` parameter

Order of merging:

    config -> inputs

In [8]:
from roux.workflow.cfgs import read_metadata 
config=read_metadata(
    f'{input_dir_path}/meta_config.yaml', # path to the meta-config
    inputs=dict(key1='modified'),
    # verbose=True, # verbose
    )
config

{'key1': 'modified',
 'key2': 'value2',
 'key3': 'interpolated from metaconfig = value2',
 'sub_config1': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
  'path': 'path/to/file1',
  'description': 'sample1.',
  'c1key1': 'value from metaconfig = modified',
  'c1key2': 'value2',
  'c1key3': 'value3'},
 'sub_config2': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
  'path': 'path/to/file2',
  'description': 'sample2.',
  'c2key1': 'value from metaconfig = modified',
  'c2key2': 'value interpolated in config1 = value from metaconfig = modified',
  'c2key3': 'value3'},
 'sub_config3': {'input_path': 'path/to/file2', 'output_path': 'output path'}}

In [9]:
## test for testbook
assert config['sub_config2']['c2key2']=='value interpolated in config1 = value from metaconfig = modified', config['sub_config2']['c2key2']
assert len(config)==6, len(config)
assert len(config['sub_config2'])==6, config['sub_config2']
print(config['sub_config2']['c2key2'])

value interpolated in config1 = value from metaconfig = modified


### Using base config and modifying config using `inputs` parameter

Order of merging:  

    base -> config -> inputs

In [10]:
from roux.workflow.cfgs import read_metadata 
config=read_metadata(
    f'{input_dir_path}/meta_config.yaml', # path to the meta-config
    inputs=dict(key1='modified'),
    config_base={'key0': 'base'}
    # verbose=True, # verbose
    )
config

{'key0': 'base',
 'key1': 'modified',
 'key2': 'value2',
 'key3': 'interpolated from metaconfig = value2',
 'sub_config1': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config1.yaml',
  'path': 'path/to/file1',
  'description': 'sample1.',
  'c1key1': 'value from metaconfig = modified',
  'c1key2': 'value2',
  'c1key3': 'value3'},
 'sub_config2': {'config_path': 'tests/input/roux_workflow_cfgs/sub_config2.yaml',
  'path': 'path/to/file2',
  'description': 'sample2.',
  'c2key1': 'value from metaconfig = modified',
  'c2key2': 'value interpolated in config1 = value from metaconfig = modified',
  'c2key3': 'value3'},
 'sub_config3': {'input_path': 'path/to/file2', 'output_path': 'output path'}}

## `cfg_run`

In [11]:
from roux.lib.log import log_dict
from roux.workflow.io import read_config
cfg=read_config(
"""
step2:
    pre:
        pms_run:
            input_path:
            output_path:
        kws_run:
            script_path:
            kernel: 
            cpus:

    pms_run:
        input_path:
        output_path:
    kws_run:
        script_path:
        kernel: 
        
    post:
        pms_run:
            input_path:
            output_path:
        kws_run:
            script_path:
            kernel: 
step1:
    tmp:
        pms_run:
            input_path:
            output_path:
        kws_run:
            script_path:
            kernel:     
""",
)
log_dict(cfg)

info:root:
step2:
  pre:
    pms_run:
      input_path: null
      output_path: null
    kws_run:
      script_path: null
      kernel: null
      cpus: null
  pms_run:
    input_path: null
    output_path: null
  kws_run:
    script_path: null
    kernel: null
  post:
    pms_run:
      input_path: null
      output_path: null
    kws_run:
      script_path: null
      kernel: null
step1:
  tmp:
    pms_run:
      input_path: null
      output_path: null
    kws_run:
      script_path: null
      kernel: null



In [12]:
from roux.workflow.cfgs import get_cfg_run
cfg_run=get_cfg_run(cfg)
log_dict(cfg_run)

info:root:
!!python/object/apply:collections.OrderedDict
- - - step2-pre
    - pms_run:
        input_path: null
        output_path: null
      kws_run:
        script_path: null
        kernel: null
        cpus: null
  - - step2
    - pms_run:
        input_path: null
        output_path: null
      kws_run:
        script_path: null
        kernel: null
  - - step2-post
    - pms_run:
        input_path: null
        output_path: null
      kws_run:
        script_path: null
        kernel: null
  - - step1-tmp
    - pms_run:
        input_path: null
        output_path: null
      kws_run:
        script_path: null
        kernel: null



In [13]:
assert cfg_run==get_cfg_run(cfg_run)

In [14]:
assert list(cfg_run.keys())==['step2-pre','step2','step2-post','step1-tmp'], list(cfg_run.keys())